## Remap events for the Attention Shift Data

This notebook assumes that you have already created a remapped template

In [1]:
import os
import datetime
from hed.tools import BidsTabularDictionary, get_file_list, get_new_dataframe, KeyMap,  HedLogger

# Variables to set for the specific dataset
bids_root_path = '/XXX/AttentionShiftWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'run')
key_columns = ["event_code", "cond_code", "focus_modality"]
target_columns = ["event_type", "attention_status", "task_role", "condition"]
final_column_order = ["onset", "duration", "sample", "sub_block",
                      "trial", "focus_modality", "event_type", "attention_status", "task_role",
                      "condition", "event_code", "cond_code"]
log_name = 'attention_shift_11_remap_events_log'
template_file = os.path.realpath(os.path.join(bids_root_path, "code/attention_shift_remap_event_template_filled.tsv"))

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Make the file list and dictionary for _events
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events", exclude_dirs=exclude_dirs)
bids_dict = BidsTabularDictionary("Event files", bids_files, entities=entities)

# Create the template
key_map = KeyMap(key_cols=key_columns, target_cols=target_columns, name='attention_shift_remap')
logger.add("overall",
           f"Creating a key map with key_columns {str(key_columns)} and target_columns {str(target_columns)}")
logger.add("overall", f"Read the template from {os.path.basename(template_file)}")
duplicates = key_map.update(template_file, allow_missing=False, keep_counts=False)
if duplicates:
    logger.add("overall", f"This mapping has {len(duplicates)} duplicate key entries", level="ERROR", also_print=True)

for key, file, rowcount, columns in bids_dict.iter_tsv_info():
    filename = file.file_path
    df_bids = get_new_dataframe(filename)
    logger.add(key, f"Remapping {os.path.basename(filename)}")
    df_new, missing = key_map.remap(filename)
    if missing:
        logger.add(key, f"The keys {str(missing)} were not in the map", level="ERROR", also_print=True)
    df_new = df_new.reindex(columns=final_column_order)
    logger.add(key, f"Reordered the columns as {str(final_column_order)}")
    df_new.to_csv(filename, sep='\t', index=False)
    logger.add(key, f"Writing {os.path.basename(filename)})")

# Print the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)
save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
attention_shift_11_remap_events_log: Level None
overall:
	[ Creating a key map with key_columns ['event_code', 'cond_code', 'focus_modality'] and target_columns ['event_type', 'attention_status', 'task_role', 'condition']]
	[ Read the template from attention_shift_remap_event_template_filled.tsv]
sub-001_run-01:
	[ Remapping sub-001_task-AuditoryVisualShift_run-01_events.tsv]
	[ Reordered the columns as ['onset', 'duration', 'sample', 'sub_block', 'trial', 'focus_modality', 'event_type', 'attention_status', 'task_role', 'condition', 'event_code', 'cond_code']]
	[ Writing sub-001_task-AuditoryVisualShift_run-01_events.tsv)]
sub-002_run-01:
	[ Remapping sub-002_task-AuditoryVisualShift_run-01_events.tsv]
	[ Reordered the columns as ['onset', 'duration', 'sample', 'sub_block', 'trial', 'focus_modality', 'event_type', 'attention_status', 'task_role', 'condition', 'event_code', 'cond_code']]
	[ Writing sub-002_task-AuditoryVisualShift_run-01_events.tsv)]
sub-003_run-01:
	[ Rem